In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [24]:
tickers = {
    'AMD': "2024/5/10",
    'NVDA': "2024/1/28",
    '2330.TW': "2024/3/21",
    'AAPL': "2023/12/5",
    '3324.TW': "2024/1/12"
}

total_df = pd.DataFrame()

for ticker, end_date in tickers.items():
    try:
        df = yf.download(ticker, start="2020-01-01", end=end_date, interval="1d")

        close_series = df['Close'].rename(ticker)

        if total_df.empty:
            total_df = close_series.to_frame()
        else:
            total_df = total_df.join(close_series, how='outer')

    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

print(total_df)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AMD']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Error fetching AMD: 'DataFrame' object has no attribute 'to_frame'


[*********************100%***********************]  1 of 1 completed

2 Failed downloads:
['NVDA', 'AMD']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Error fetching NVDA: 'DataFrame' object has no attribute 'to_frame'


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['2330.TW']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Error fetching 2330.TW: 'DataFrame' object has no attribute 'to_frame'


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Error fetching AAPL: 'DataFrame' object has no attribute 'to_frame'


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['3324.TW']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Error fetching 3324.TW: 'DataFrame' object has no attribute 'to_frame'
Empty DataFrame
Columns: []
Index: []


## risk reward ratio

In [ ]:
risk_reward_results = {}

for ticker in total_df.columns:
    series = total_df[ticker].dropna()

    net_profit = series.iloc[-1] - series.iloc[0]
    
    running_max = series.cummax()
    drawdown = (series - running_max) / running_max
    max_drawdown = drawdown.min()
    
    if max_drawdown != 0:
        risk_reward = net_profit / abs(max_drawdown)
        risk_reward_results[ticker] = {
            'Net Profit': net_profit,
            'Max Drawdown': max_drawdown,
            'Risk/Reward Ratio': risk_reward
        }
    else:
        risk_reward_results[ticker] = {
            'Net Profit': net_profit,
            'Max Drawdown': max_drawdown,
            'Risk/Reward Ratio': None
        }

risk_reward_df = pd.DataFrame(risk_reward_results).T
risk_reward_df = risk_reward_df.sort_values(by='Risk/Reward Ratio', ascending=False)

print(risk_reward_df)

KeyError: 'Risk/Reward Ratio'

## seconed

In [ ]:
email_df = pd.DataFrame({
    'Analyst': ['Julia', 'William', 'Jason', 'Cary'],
    'Email': [
        'julia.jokic@mmafundmgmt.com',
        'william.wang@mmafundmgmt.com',
        'jason.yang@mmafundmgmt.com',
        'cary.chen@mmafundmgmt.com'
    ]
})

df['Exceed Limit'] = (df['Current Price'] >= df['Upside']) | (df['Current Price'] <= df['Downside'])

alert_df = df[df['Exceed Limit']].merge(email_df, on='Analyst', how='left')

for _, row in alert_df.iterrows():
    print(f"""
[提醒通知] 股票：{row['Stock']}
目前價格：{row['Current Price']}
已經{'超過上限' if row['Current Price'] >= row['Upside'] else '低於下限'}（預測區間：{row['Downside']} ~ {row['Upside']}）
分析師：{row['Analyst']}，Email：{row['Email']}
""")

alert_df.to_excel("Quant_Problem1_Alert_List.xlsx", index=False)

KeyError: 'Current Price'

## third

In [ ]:
risk_reward_df = pd.read_csv("/Users/xinc./Documents/GitHub/Quant/MMA/Quant_Problem1_result.csv")
top3_stocks = risk_reward_df.sort_values(by = 'Risk Reward Ratio', ascending = False)

top3_prices = total_df[top3_stocks]

top3_prices.to_csv("top3_stocks_prices.csv")